将DAIC的数据集中的阳性样本按10000的片段重新过采样， 对于阴性样本，只保留中间10000时长片段， 重新组成数据集， 送到H2OFlow里面运行AutoML.

In [3]:

import pandas as pd
import numpy as np

In [4]:
feature_names=['F0','VUV','NAQ','QOQ','H1H2','PSP','MDQ','peakSlope','Rd', 
    'Rd_conf','creak','MCEP_0','MCEP_1','MCEP_2','MCEP_3','MCEP_4','MCEP_5', 
    'MCEP_6','MCEP_7','MCEP_8','MCEP_9','MCEP_10','MCEP_11','MCEP_12', 
    'MCEP_13','MCEP_14','MCEP_15','MCEP_16','MCEP_17','MCEP_18', 
    'MCEP_19','MCEP_20','MCEP_21','MCEP_22','MCEP_23','MCEP_24',
    'HMPDM_0','HMPDM_1','HMPDM_2','HMPDM_3','HMPDM_4','HMPDM_5', 
    'HMPDM_6','HMPDM_7','HMPDM_8','HMPDM_9','HMPDM_10','HMPDM_11','HMPDM_12', 
    'HMPDM_13','HMPDM_14','HMPDM_15','HMPDM_16','HMPDM_17','HMPDM_18', 
    'HMPDM_19','HMPDM_20','HMPDM_21','HMPDM_22','HMPDM_23','HMPDM_24',
    'HMPDD_0','HMPDD_1','HMPDD_2','HMPDD_3','HMPDD_4','HMPDD_5', 
    'HMPDD_6','HMPDD_7','HMPDD_8','HMPDD_9','HMPDD_10','HMPDD_11','HMPDD_12'];

stat_feature_names = list(map(lambda n : [f'{n}_mean',f'{n}_std',f'{n}_min',f'{n}_25',f'{n}_50',f'{n}_75',f'{n}_max'], feature_names))
M,N = np.shape(stat_feature_names)
stat_feature_names = np.reshape(stat_feature_names,(M*N))


In [5]:

# 将时序数据聚集到一组统计变量，mean, ...


y_train_df = pd.read_csv('/Users/jingming/DataSets/COVAREP/y_Train.tsv',sep='\t',header=None, names=['id','label'])
y_train = y_train_df.iloc[:,1]

y_test_df = pd.read_csv('/Users/jingming/DataSets/COVAREP/y_Test.tsv',sep='\t',header=None, names=['id','label'])
y_test = y_test_df.iloc[:,1]

y_df = pd.concat([y_train_df, y_test_df])


In [64]:
y_df.iloc[:,1]

0     0
1     0
2     0
3     0
4     0
     ..
42    0
43    0
44    0
45    0
46    0
Name: label, Length: 154, dtype: int64

In [10]:
row_list = []
y_list = []

window_size = 10000

for index,row in y_df.iterrows():
    df = pd.read_csv((f'/Users/jingming/DataSets/COVAREP/{row["id"]}_COVAREP.csv'),sep=',',header=None, names=feature_names)
    df = df[df['VUV']==1]
    n = df.to_numpy().shape[0]
    if row['label'] == 1:
        for j in range(0,int(n/window_size)):
            # df = np.transpose(df.to_numpy())
            # X_train.append(np.transpose(df[j*window_size:(j+1)*window_size].to_numpy()))
            a = df[j*window_size:(j+1)*window_size].describe().iloc[1:].to_numpy()
            row_list.append(np.reshape(a,(a.size),order='F'))
            y_list.append(1)

    if row['label'] == 0:
        pad = int((n-window_size)/2)

        a = df[pad:(pad+window_size)].describe().iloc[1:].to_numpy()
        row_list.append(np.reshape(a,(a.size),order='F'))
        
        y_list.append(0)

X_df = pd.DataFrame(data = row_list, columns=stat_feature_names)
X_y_df = X_df.assign(y=y_list)

In [11]:
X_y_df.to_csv('DAIC_stat_features_X_y_w10000_resample.csv')

In [69]:
X_y_df.head()

,F0_mean,F0_std,F0_min,F0_25,F0_50,F0_75,F0_max,VUV_mean,VUV_std,VUV_min,...,HMPDD_11_75,HMPDD_11_max,HMPDD_12_mean,HMPDD_12_std,HMPDD_12_min,HMPDD_12_25,HMPDD_12_50,HMPDD_12_75,HMPDD_12_max,label
0,199.187302,43.504720,93.0,173.0,189.0,212.0,386.0,1.0,0.0,1.0,...,0.124040,0.77371,0.081089,0.161551,-0.49488,-0.015867,0.10117,0.201270,0.64151,0
1,218.017081,48.128494,100.0,187.5,205.0,242.0,416.0,1.0,0.0,1.0,...,0.195580,0.71642,0.127041,0.163414,-0.81961,0.040083,0.16172,0.242205,0.73451,0
2,126.224182,39.950262,58.0,103.0,114.0,135.5,254.0,1.0,0.0,1.0,...,-0.202420,0.61953,-0.182274,0.141901,-0.49445,-0.278567,-0.21172,-0.126240,0.68991,0
3,90.751698,23.661682,50.0,76.0,84.0,96.5,174.0,1.0,0.0,1.0,...,-0.288077,0.46514,-0.216009,0.110370,-0.50076,-0.295715,-0.23695,-0.153985,0.23156,0
4,118.682533,40.009358,50.0,88.5,105.5,138.5,226.0,1.0,0.0,1.0,...,-0.221880,0.55367,-0.185922,0.151518,-0.56354,-0.291600,-0.22437,-0.124507,0.56220,0


In [70]:
X_y_df.to_csv('DAIC_stat_features_X_y.csv')

In [50]:
df = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]})

# a = [[0,1],
#     [2,3],
#     [4,5]]

print(df.describe().iloc[1:])



        A    B
mean  2.0  5.0
std   1.0  1.0
min   1.0  4.0
25%   1.5  4.5
50%   2.0  5.0
75%   2.5  5.5
max   3.0  6.0
